## Tell the notebook where to access cloud storage, if you're using it

In [1]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
ee.Initialize()
imc = ee.ImageCollection('users/grahamrjeffries/brazil_lc_morgen')
bioma = ee.FeatureCollection('users/daviddralle/bioma')

# Reclassify lc_morgen
fromlist = [0, 1, 2, 4, 5, 6]
tolist = [5, 1, 1, 1, 3, 3]
def remapcustom(image):
    return image.remap(fromlist, tolist)
imc = imc.map(remapcustom).map(lambda image: image.rename(['lc']))

In [2]:
df = pd.read_csv( gcs + '/rally_points.csv')
df.head()

,x,y,year,id
0,-59.242406,-13.539276,2006,1
1,-59.119188,-13.427609,2006,2
2,-59.084533,-13.469965,2006,3
3,-59.076832,-13.558529,2006,4
4,-59.057579,-13.650943,2006,5


In [101]:
whatToSample = 'lc'
scl = 30
regionsId = 'id'

years = df.year.values
ft = []

for year in np.unique(years):
    toSample = imc.filterDate(str(year), str(year+1))
    temp_pts = df.loc[df.year==year]
    xy = [item for item in zip(temp_pts.x.values, temp_pts.y.values)]
    ids = temp_pts.id.values
    pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(xy[i][0],xy[i][1]))
                            .set('id', ids[i])
                            for i in range(len(xy))])
    regions = pts
    triplets = toSample.map(lambda image: 
                       ee.Image(image.select(whatToSample)).
                       reduceRegions(regions,reducer=ee.Reducer.first(),scale=scl)
                       .map(lambda f: f.set(whatToSample,f.get('first')))
                      ).flatten()

    print(triplets.first().getInfo())
    name = whatToSample + '_' + str(year) + '_'
    ee.batch.Task.start(
        ee.batch.Export.table(
            triplets, 
            config={'outputBucket':'agroserve_climate','outputPrefix':name,'fileFormat':'geoJSON'}
        )
    )

    
# path = triplets.getDownloadURL(filetype='json')
# print(path)

    

{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.242406486176215, -13.53927608779361]}, u'type': u'Feature', u'id': u'lc_2006_0', u'properties': {u'lc': 1, u'id': 1, u'first': 1}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.288613316751174, -13.78186368887177]}, u'type': u'Feature', u'id': u'lc_2007_0', u'properties': {u'lc': 3, u'id': 891, u'first': 3}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.584333333333305, -13.5499722222222]}, u'type': u'Feature', u'id': u'lc_2008_0', u'properties': {u'lc': 5, u'id': 1659, u'first': 5}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.304015593609506, -13.75876010781671]}, u'type': u'Feature', u'id': u'lc_2009_0', u'properties': {u'lc': 3, u'id': 2709, u'first': 3}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.816141299148576, -13.55082787832114]}, u'type': u'Feature', u'id': u'lc_2010_0', u'properties': {u'

In [102]:
# reclass mapbiomas
mb = ee.Image('projects/mapbiomas-workspace/public/collection2_3/mapbiomas_collection23_integration_v1')
fromlist = range(1,28)
tolist = len(fromlist)*[2]
tolist[17] = 1
tolist[20] = 1
def remapcustom(image):
    return image.remap(fromlist, tolist)

whatToSample = 'mb'
scl = 30
regionsId = 'id'

years = df.year.values
ft = []

for year in np.unique(years):
    toSample = ee.ImageCollection([ee.Image(mb.select('classification_' + str(year))).remap(fromlist, tolist).rename(['mb'])])
    temp_pts = df.loc[df.year==year]
    xy = [item for item in zip(temp_pts.x.values, temp_pts.y.values)]
    ids = temp_pts.id.values
    pts = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(xy[i][0],xy[i][1]))
                            .set('id', ids[i])
                            for i in range(len(xy))])
    regions = pts
    triplets = toSample.map(lambda image: 
                       ee.Image(image.select(whatToSample)).
                       reduceRegions(regions,reducer=ee.Reducer.first(),scale=scl)
                       .map(lambda f: f.set(whatToSample,f.get('first')))
                      ).flatten()
    print(triplets.first().getInfo())
    name = whatToSample + '_' + str(year) + '_'
    ee.batch.Task.start(
        ee.batch.Export.table(
            triplets, 
            config={'outputBucket':'agroserve_climate','outputPrefix':name, 'fileFormat':'geoJSON'}
        )
    )    


{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.242406486176215, -13.53927608779361]}, u'type': u'Feature', u'id': u'mapbiomas_collection23_integration_v1_0', u'properties': {u'mb': 1, u'id': 1, u'first': 1}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.288613316751174, -13.78186368887177]}, u'type': u'Feature', u'id': u'mapbiomas_collection23_integration_v1_0', u'properties': {u'mb': 1, u'id': 891, u'first': 1}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.584333333333305, -13.5499722222222]}, u'type': u'Feature', u'id': u'mapbiomas_collection23_integration_v1_0', u'properties': {u'mb': 2, u'id': 1659, u'first': 2}}
{u'geometry': {u'type': u'Point', u'geodesic': True, u'coordinates': [-59.304015593609506, -13.75876010781671]}, u'type': u'Feature', u'id': u'mapbiomas_collection23_integration_v1_0', u'properties': {u'mb': 1, u'id': 2709, u'first': 1}}
{u'geometry': {u'type': u'Point', u'geodesic': True, 

In [116]:
import glob
lcs = pd.concat([gp.read_file(item) for item in glob.glob(gcs + '/lc*.geojson')])
mb = pd.concat([gp.read_file(item) for item in glob.glob(gcs + '/mb*.geojson')])
df = mb.merge(lcs, how='inner', on='id')
df['mb_lc'] = df.mb*df.lc
df = df[['mb_lc', 'geometry_x', 'id']]
df['lat'] = df['geometry_x'].apply(lambda row: row.y)
df['lon'] = df['geometry_x'].apply(lambda row: row.x)
df.to_csv('/mnt/mb_lc.csv')